In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow import keras
from keras.layers import *
import numpy as np

In [3]:
from google.colab import drive

drive.mount('/content/drive')
RunningInCOLAB = 'google.colab' in str(get_ipython())

path = "./data_collection/snake2_output.csv"
if RunningInCOLAB: path = "/content/drive/MyDrive/24Winter/cs320-w24/cs320-final-project/data_collection/snake2_game_output_1.csv"
df_import = pd.read_csv(path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
# GLOBAL VARIABLES
USE_DUMMY_FEATURES = False

In [25]:
df_import = pd.read_csv(path)
df_import.head()
df = df_import
df.head()

,obstacleL,obstableF,obstacleR,direction_choice,game_over
0,0,0,0,2,0
1,0,0,0,2,0
2,0,0,0,1,0
3,0,0,0,1,0
4,0,0,0,0,0


In [26]:
# selecting better training set:
df_game_over = df[df['game_over'] == 1]
df_game_not_over = df[df['game_over'] == 0]
df_game_not_over_sample = df_game_not_over.sample(n=500, random_state=42)
df_game_not_over_sample

# this df will have more equal amounts of dead/alive examples
df_sample = pd.concat([df_game_over, df_game_not_over_sample], ignore_index=True)


In [27]:
dummy_columns = None

# df = df_sample

if USE_DUMMY_FEATURES:
  dummy_columns = pd.get_dummies(df['direction_choice'], prefix='direction_choice')

def dir_num_to_array(direction: int):
  t = [0,0,0]
  t[direction] = 1
  return t

In [28]:
df['direction_choice'] = df['direction_choice'] - 1
df

,obstacleL,obstableF,obstacleR,direction_choice,game_over
0,0,0,0,1,0
1,0,0,0,1,0
2,0,0,0,0,0
3,0,0,0,0,0
4,0,0,0,-1,0
...,...,...,...,...,...
42469,0,0,0,-1,0
42470,0,0,0,-1,0
42471,0,0,0,1,0
42472,0,1,0,-1,0


In [29]:
dummy_columns

In [30]:
if USE_DUMMY_FEATURES:
  df = pd.concat([df, dummy_columns], axis=1)
  df.drop('direction_choice', inplace=True, axis=1)

In [31]:
df.head()

,obstacleL,obstableF,obstacleR,direction_choice,game_over
0,0,0,0,1,0
1,0,0,0,1,0
2,0,0,0,0,0
3,0,0,0,0,0
4,0,0,0,-1,0


In [32]:
X = df.drop('game_over', axis=1).to_numpy()
y = df['game_over'].to_numpy()
print(X.shape, "\n", y.shape)

X_feature_shape = X.shape[1]
X_feature_shape

(42474, 4) 
 (42474,)


4

In [33]:
# create testing and training data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [34]:
# init model
model = keras.Sequential()

# input layer
model.add(Input(shape=(X_feature_shape,)))
model.add(Dense(units=(5), activation='sigmoid'))
# model.add(Dense(units=(5)))
model.add(Dense(units=(1), activation='sigmoid'))

optimizer = keras.optimizers.Adam(learning_rate=1e-1)
model.compile(optimizer=optimizer, loss='mean_squared_error', metrics=['accuracy'])

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 5)                 25        
                                                                 
 dense_5 (Dense)             (None, 1)                 6         
                                                                 
Total params: 31 (124.00 Byte)
Trainable params: 31 (124.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [36]:
model.fit(X_train, y_train, epochs=4, batch_size=32)

Epoch 1/4
930/930 [==============================] - 4s 4ms/step - loss: 1.5146e-05 - accuracy: 1.0000
Epoch 2/4
930/930 [==============================] - 3s 4ms/step - loss: 7.7739e-06 - accuracy: 1.0000
Epoch 3/4
930/930 [==============================] - 4s 4ms/step - loss: 4.3644e-06 - accuracy: 1.0000
Epoch 4/4
930/930 [==============================] - 3s 3ms/step - loss: 2.5919e-06 - accuracy: 1.0000


In [18]:
model.save("/content/drive/MyDrive/24Winter/cs320-w24/cs320-final-project/models/snake2_possiblemvp_functional1.keras")

In [37]:
model.evaluate(X_train, y_train)

930/930 [==============================] - 2s 2ms/step - loss: 1.9653e-06 - accuracy: 1.0000


[1.9653173239930766e-06, 1.0]

In [38]:
y_preds = model.predict(X_test)

y_preds[y_preds > 0.5] = 1
y_preds[y_preds <= 0.5] = 0

399/399 [==============================] - 1s 2ms/step


In [39]:
# overall accuracy
y_test_reshape = y_test.reshape((y_test.shape[0], 1))

print("Overall Accuracy:")
len(y_preds[np.where(y_preds == y_test_reshape)])/len(y_preds)

Overall Accuracy:


1.0

In [40]:
# accuracy where game_over == 1
y_preds_gameover_subset = y_preds[X_test[:,3] == 1]
y_test_subset = y_test[X_test[:,3] == 1]
y_test_subset = y_test_subset.reshape((y_test_subset.shape[0], 1))

len(y_preds_gameover_subset[y_preds_gameover_subset == y_test_subset])/len(y_test_subset)

1.0

In [41]:
y_test

array([0, 0, 0, ..., 0, 0, 0])

In [42]:
X_test[2]

array([ 0,  0,  0, -1])

In [43]:
model.predict(X_test[1].reshape((1,4)))

1/1 [==============================] - 0s 42ms/step


array([[0.0005905]], dtype=float32)

In [54]:
obs = [0,0,1]
direction = -1
input = np.concatenate([obs, [direction]]).reshape((1,4))

print(input)


model.predict(input)

[[ 0  0  1 -1]]
1/1 [==============================] - 0s 30ms/step


array([[0.00061874]], dtype=float32)